In [29]:
import pandas as pd
import dask.dataframe as dd
from pathlib import Path
from sqlalchemy import create_engine, text

In [30]:
#Reading in the source file for the project
file_path = 'Resources/wdl_data_en_reduced.csv'
# Specifying encoding as'ISO-8859-1' to read file
loc_df = pd.read_csv(file_path, encoding='utf-8')

C:\Users\Cait\AppData\Local\Temp\ipykernel_29956\2306786982.py:4: DtypeWarning: Columns (0,20,33,34,36,37,41,42,43,49,52,54,55,62,63,66,69,70,71,72,74,76,81,82,84,85,86,87) have mixed types. Specify dtype option on import or set low_memory=False.
  loc_df = pd.read_csv(file_path, encoding='utf-8')


In [31]:
# dataframe for all the languages
lang_df = pd.DataFrame(loc_df['Language'])
lang_df.head()

,Language
0,NaN
1,NaN
2,Japanese
3,Japanese
4,Spanish


In [32]:
# split the entries into separate columns because some entries have more than one language
lang_df_split = lang_df['Language'].str.split('|', expand=True)
lang_df_split.head()

,0,1,2,3,4,5,6,7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Japanese,None,None,None,None,None,None,None
3,Japanese,None,None,None,None,None,None,None
4,Spanish,None,None,None,None,None,None,None


In [33]:
# get a merged view of the data by turning each column into its own dataframe
lang_df_col0 = lang_df_split.iloc[:,0].rename('Lang')
lang_df_col1 = lang_df_split.iloc[:,1].rename('Lang')
lang_df_col2 = lang_df_split.iloc[:,2].rename('Lang')
lang_df_col3 = lang_df_split.iloc[:,3].rename('Lang')
lang_df_col4 = lang_df_split.iloc[:,4].rename('Lang')
lang_df_col5 = lang_df_split.iloc[:,5].rename('Lang')
lang_df_col6 = lang_df_split.iloc[:,6].rename('Lang')
lang_df_col7 = lang_df_split.iloc[:,7].rename('Lang')

In [34]:
# get all of the languages entries with their item id
# drop all of the null values
lang_df_col0 = pd.DataFrame(lang_df_col0)
lang_df_col0['item_id'] = lang_df_col0.index
lang_df_col0 = lang_df_col0.dropna()
lang_df_col1 = pd.DataFrame(lang_df_col1)
lang_df_col1['item_id'] = lang_df_col1.index
lang_df_col1 = lang_df_col1.dropna()
lang_df_col2 = pd.DataFrame(lang_df_col2)
lang_df_col2['item_id'] = lang_df_col2.index
lang_df_col2 = lang_df_col2.dropna()
lang_df_col3 = pd.DataFrame(lang_df_col3)
lang_df_col3['item_id'] = lang_df_col3.index
lang_df_col3 = lang_df_col3.dropna()
lang_df_col4 = pd.DataFrame(lang_df_col4)
lang_df_col4['item_id'] = lang_df_col4.index
lang_df_col4 = lang_df_col4.dropna()
lang_df_col5 = pd.DataFrame(lang_df_col5)
lang_df_col5['item_id'] = lang_df_col5.index
lang_df_col5 = lang_df_col5.dropna()
lang_df_col6 = pd.DataFrame(lang_df_col6)
lang_df_col6['item_id'] = lang_df_col6.index
lang_df_col6 = lang_df_col6.dropna()
lang_df_col7 = pd.DataFrame(lang_df_col7)
lang_df_col7['item_id'] = lang_df_col7.index
lang_df_col7 = lang_df_col7.dropna()

In [35]:
# combine all of the language column dataframes into one 
lang_merged_df = pd.merge(lang_df_col0, lang_df_col1, how='outer')
lang_merged_df = pd.merge(lang_merged_df, lang_df_col2, how='outer')
lang_merged_df = pd.merge(lang_merged_df, lang_df_col3, how='outer')
lang_merged_df = pd.merge(lang_merged_df, lang_df_col4, how='outer')
lang_merged_df = pd.merge(lang_merged_df, lang_df_col5, how='outer')
lang_merged_df = pd.merge(lang_merged_df, lang_df_col6, how='outer')
lang_merged_df = pd.merge(lang_merged_df, lang_df_col7, how='outer')
lang_merged_df.head()

,Lang,item_id
0,Japanese,2
1,Japanese,3
2,Spanish,4
3,Dutch,5
4,Latin,6


In [36]:
# stack all of the columns into one to filter out just the unique values
lang_df_stack = lang_df_split.stack()
lang_df_stack

2      0    Japanese
3      0    Japanese
4      0     Spanish
5      0       Dutch
       1     Spanish
              ...   
23857  1      Pushto
23858  0     Persian
       1      Pushto
23859  0     Persian
       1      Pushto
Length: 18376, dtype: object

In [37]:
# convert the stack into a dataframe
lang_df_stack = pd.DataFrame(lang_df_stack)
lang_df_stack

0
2     0  Japanese
3     0  Japanese
4     0   Spanish
5     0     Dutch
      1   Spanish
...           ...
23857 1    Pushto
23858 0   Persian
      1    Pushto
23859 0   Persian
      1    Pushto

[18376 rows x 1 columns]

In [38]:
# rename the '0' column to 'Lang'
lang_df_stack = lang_df_stack.iloc[:,0].rename('Lang')

In [39]:
lang_df_stack = pd.DataFrame(lang_df_stack)
lang_df_stack

Lang
2     0  Japanese
3     0  Japanese
4     0   Spanish
5     0     Dutch
      1   Spanish
...           ...
23857 1    Pushto
23858 0   Persian
      1    Pushto
23859 0   Persian
      1    Pushto

[18376 rows x 1 columns]

In [40]:
# show just the unique language values
lang_df_unique = lang_df_stack['Lang'].unique()
lang_df_unique

array(['Japanese', 'Spanish', 'Dutch', 'Latin', 'English', 'Russian',
       'Portuguese', 'German', 'Arabic', 'French', 'Chinese', 'Tagalog',
       'Aleut', 'Classical Nahuatl', 'Ottoman Turkish (1500-1928)',
       'Marathi', 'Sanskrit', 'Persian', 'Naxi', 'Omaha-Ponca', 'Italian',
       'Chagatai', 'Armenian', 'Bulgarian', 'Modern Greek (1453-)',
       'Belarusian', 'Polish', 'Georgian', 'Swedish', 'Nyoro',
       'Guerrero Nahuatl', 'Eastern Huasteca Nahuatl', 'Hebrew',
       'Central Nahuatl', 'Ladino', 'Old French (842-ca. 1400)', 'Mossi',
       'Dazaga', 'Czech', 'Serbo-Croatian', 'Church Slavic', 'Tuvalu',
       'Hindi', 'Burmese', 'Turkish', 'Slovak', 'Danish', 'Norwegian',
       'Hungarian', 'Javanese', 'Batak', 'Afrikaans', 'Tibetan', 'Manchu',
       'Mongolian', 'Ancient Greek (to 1453)', 'Tamil',
       'Northwest Oaxaca Mixtec', 'Kurdish', 'Scots',
       'Judeo-Yemeni Arabic', 'Judeo-Arabic', 'Mikasuki', 'Welsh',
       'Middle Cornish', 'Syriac', 'Middle High Ge

In [41]:
# put the unique languages into a dataframe
unique_lang = pd.DataFrame()
unique_lang['Lang'] = lang_df_unique
unique_lang

,Lang
0,Japanese
1,Spanish
2,Dutch
3,Latin
4,English
...,...
108,Macedonian
109,Geez
110,Pushto
111,Old High German (ca. 750-1050)


In [42]:
# make a column that stores the index to act as a key
unique_lang['lang_id'] = unique_lang.index

In [43]:
# Export final language dataframe to CSV file
unique_lang.to_csv('Resources/languages_id.csv', index=False)

In [44]:
# the final unique item dataframe
unique_lang.head()

,Lang,lang_id
0,Japanese,0
1,Spanish,1
2,Dutch,2
3,Latin,3
4,English,4


In [51]:
# create a dataframe to represent the relationship between item_id and lang_id to tie the item and language tables together
lang_item = lang_merged_df.merge(unique_lang,how='right')
lang_item

,Lang,item_id,lang_id
0,Japanese,2,0
1,Japanese,3,0
2,Japanese,7,0
3,Japanese,8,0
4,Japanese,9,0
...,...,...,...
18371,Pushto,23859,110
18372,Old High German (ca. 750-1050),11811,111
18373,Old High German (ca. 750-1050),23741,111
18374,Sichuan Yi,11889,112


In [52]:
# drop the 'Lang' column
lang_item.drop(columns='Lang',inplace=True)

In [55]:
lang_item.head()

,item_id,lang_id
0,2,0
1,3,0
2,7,0
3,8,0
4,9,0


In [56]:
# export the relationship data to a .csv file
lang_item.to_csv('Resources/lang_item.csv', index=False)